## This code apply MetalTiger to produce normalized text

In [ ]:
import os
src_dir = 'all_text'
tar_dir = 'all_speech'
i=0
count = 1
with os.scandir(src_dir) as files:
    
    for file in files:
        
        nam= str(file)
        file=nam[11:-2]
        nam='speech-form-'+ nam[21:-2]
        
        file= 'all_text/'+file
        #print(file)
        #print(nam)
        output_file =  'all_speech/'+nam
        
        if i < count:
            
            print('started: ',nam)

            #! ./MetalTiger transform -config metaltiger-master/sample_models/lm_clean/en_US/spokenform.toml -input $file \
            #        >$output_file
        

        print('Done upto', nam, i)
        i +=1
    print('Complete!')

### This code apply metal tiger's text_cleaning.toml for text cleaning  

In [ ]:
import os
src_dir = 'all_text'
i=0
count = -1
with os.scandir(src_dir) as files:
    
    for file in files:
        
        nam= str(file)
        file=nam[11:-2]
        
        
        
        #print(file)
        #print(nam)
        output_file =  'all_text_cleaned/'+file
        file = 'all_text/'+file
        if i > count:
            
            print('started: ',file)

           # ! ./MetalTiger transform -config metaltiger-master/sample_models/lm_clean/en_US/text_cleaning.toml -input $file \
           #         >$output_file
        

        print('Done upto', file, i)
        i +=1
    print('Complete!')

#### Applying sclite to produce comparsion

In [ ]:
import os
src_dir = 'all_text_cleaned'
i=0
count = -1
with os.scandir(src_dir) as files:
    
    for file in files:
        
        nam= str(file)
        file=nam[21:-2]

        ref =  'all_text_cleaned/text-form-'+file
        hyp = 'all_speech/speech-form-'+file
        tar = 'Comparison-files/comp-'+file
        if i > count:
            
            
            !cat $ref | tr '\n' ' ' > single-test-out/ref.txt
            !echo "(gar)" >> single-test-out/ref.txt
            !cat $hyp | tr '\n' ' ' > single-test-out/hyp.txt
            !echo "(gar)" >> single-test-out/hyp.txt

            !./sclite -i swb -r single-test-out/ref.txt -h single-test-out/hyp.txt -o pralign sum -l 140 -O single-test-out
            !cp single-test-out/hyp.txt.pra $tar
            
            
        
        
        print('Done upto', file, i)
        i +=1
    print('Complete!')

In [ ]:
ref = 'all_text_cleaned/text-form-VS-849-ESQMIA-J7147578.txt'
hyp = 'all_speech/speech-form-VS-849-ESQMIA-J7147578.txt'

!cat $ref | tr '\n' ' ' > single-test-out/ref.txt
!echo "(gar)" >> single-test-out/ref.txt
!cat $hyp | tr '\n' ' ' > single-test-out/hyp.txt
!echo "(gar)" >> single-test-out/hyp.txt

!./sclite -i swb -r single-test-out/ref.txt -h single-test-out/hyp.txt -o pralign sum -l 140 -O single-test-out

## Preparing final data from comparison files V1.0

In [ ]:
def findText(line,startpos,l):
    
    t=''
    words = line[startpos:].split(' ')
    cle_words= []
    
    i=0
    
    for word in words:
        #if i < 15:
            if word != '':
                cle_words.append(word)
           # i +=1
       # else:
          #  break

    for i in range(l):
        t =t+ cle_words[i]+ ' '
        
    #print(t)
    return t


In [ ]:



def dataExtractor(file):
    
    #step-1 make a single string of speech,text and eval from a whole text file
    import re
    with open(file,'r') as f:
        lines=f.readlines()

    linecount =0

    for line in lines:
        linecount +=1

    print(linecount)
    SpeechLine = ''
    TextLine = ''
    EvalLine = ''

    i=16          #avoid first 16 lines

    while True:

        if (i+3) < linecount: 

            TextLine += lines[i][8:]
            SpeechLine += lines[i+1][8:]
            EvalLine += lines[i+2][8:]

            i +=4
        else: break   
    SpeechLine = re.sub('\n','', SpeechLine)
    TextLine = re.sub('\n','',TextLine)
    EvalLine = re.sub('\n','',EvalLine)






    #step-2 find location of 'I',"S" inside evalLIne

    template = []

    for i in range(len(EvalLine)):
        if EvalLine[i] == 'I' or EvalLine[i] == 'S':
            template.append(i)



    #step-3: start with a 'S' find next 'S' and check if it is the next word of sppechline. if yes then find next else it's the end 



    textData = []
    speechData = []

    i=0

    while True:

        if EvalLine[template[i]] == 'S':

            startpos_text = template[i]
            len_text= 1

            curr_s = i
            b= i+1

            while True:
                if b > (len(template)-1):
                    break
                elif EvalLine[template[b]] == 'S':

                    next_s= b

                    #slice_from_curr_s
                    slices_from_curr = SpeechLine[template[curr_s]:].split(' ')
                    slices_from_next =  SpeechLine[template[next_s]:].split(' ')


                    if slices_from_curr[1]== slices_from_next[0] :
                        curr_s = b
                        len_text +=1
                        b +=1
                        continue
                    else:
                        break
                else:
                    break


            a=i-1
            while True:
                if a < 0:
                    break

                elif EvalLine[template[a]] == 'S':
                    break


                elif EvalLine[template[a]] == 'I':
                    a= a-1
                    continue

            startpos_speech = template[(a+1)]

            len_speech = b-a-1

            t= findText(TextLine,startpos_text,len_text)
            s= findText(SpeechLine,startpos_speech, len_speech)

            textData.append(t)
            speechData.append(s)    
          
            i = b

        else:
            i +=1

        if i > (len(template)-1):
            break

   
    return textData, speechData


In [ ]:
file = 'Comparison-files/comp-VS-1035-ESQMIA-J7374393.txt'

import os
parenText = []
parenSpeech = []
dir= '../formatter/Comparison-files'



with os.scandir(dir) as entries:
    for entry in entries:
        
        text, speech = dataExtractor(entry)
        parenText += text
        parenSpeech += speech
        
        
import pandas as pd
df = pd.DataFrame()
df['Text-form']= parenText
df['Speech-form']= parenSpeech      
        

In [ ]:
#df.to_csv('FinalData.csv')
for i in range(1,2,1):
    print(i)

## Preparing final data from comparison files V 2.0

Algorithm: Declare two list one for storing text form and other for speech form. we will go forward by comparing two word lines if we see match then we add them in both list. If we find mismatch that means it's the start of the normalization and will store the words in different variable untill we get match words again. Once we get matched word we are end of the normalized words. We will now add the words which was unmatched as a single sample to text form and speech form.

In [ ]:



def LinesExtractor(file):
    
    #step-1 make a single string of speech,text and eval from a whole text file
    import re
    with open(file,'r') as f:
        lines=f.readlines()

    linecount =0

    for line in lines:
        linecount +=1

        
    SpeechLine = ''
    TextLine = ''
    EvalLine = ''

    i=16          #avoid first 16 lines

    while True:

        if (i+3) < linecount: 

            TextLine += lines[i][8:]
            SpeechLine += lines[i+1][8:]
            EvalLine += lines[i+2][8:]

            i +=4
        else: break   
    SpeechLine = re.sub('\n','', SpeechLine)
    TextLine = re.sub('\n','',TextLine)
    EvalLine = re.sub('\n','',EvalLine)
    
    
    return TextLine, SpeechLine, EvalLine

In [ ]:
def finalizer(t,s):   #prepares text and speech coloumn from text and speech line

    speech_form = []
    text_form = []

    text_words=  t.split(' ')
    temp = []
    for word in text_words:
        if word != '':
            temp.append(word)

    text_words= temp 

    speech_words= s.split(' ')
    temp = []
    for word in speech_words:
        if word != '':
            temp.append(word)
    speech_words= temp

    is_match = 1


    text_sample= ''
    speech_sample= ''

    for i in range(len(text_words)):

        if text_words[i] == speech_words[i]:

            if is_match == 0:

                speech_form.append(speech_sample)
                text_form.append(text_sample)

            is_match = 1
            text_sample =''   #reinitialize text_sample and speech_sample after one update
            speech_sample= ''
            speech_form.append(text_words[i])
            text_form.append(text_words[i])

        else:
            is_match = 0 
            speech_sample  += ' '+speech_words[i]
            if re.match('\*',text_words[i]):
                continue
            text_sample    += ' '+ text_words[i]



 

    return text_form,speech_form


In [ ]:

import os
import pandas as pd
df = pd.DataFrame()

parenText = []
parenSpeech = []
dir= '../formatter/Comparison-files'

with os.scandir(dir) as entries:
    for entry in entries:
        
        avoid = str(entry)[11:-2]
       
        if avoid == '.ipynb_checkpoints':
            continue
       
        t,s,e = LinesExtractor(entry)
        text, speech= finalizer(t,s)
        if len(text) != len(speech):
            print(len(text),len(speech))
            print(avoid)
            continue
        
        parenText += text
        parenSpeech += speech
        
for i in range(len(parenText)):
    parenText[i] = parenText[i].lower()
    parenSpeech[i]= parenSpeech[i].lower()

df['Text-form']= parenText
df['Speech-form']= parenSpeech      
        



In [ ]:
df.to_csv('FinalDatasetWithContext.csv',index= False)

In [ ]:
a= 'hello BRO'
print(a.lower())

In [ ]:

text_words=  t.split(' ')
temp = []
for word in text_words:
    if word != '':
        temp.append(word)

text_words= temp 

speech_words= s.split(' ')
temp = []
for word in speech_words:
    if word != '':
        temp.append(word)
speech_words= temp


for i in range(len(speech_words)):
    if text_words[i] != speech_words[i]:
        print(text_words[i],'--', speech_words[i])
        print('at: ',i)

In [ ]:
print(text[2730:3000])
print(speech[2730:3000])

In [ ]:
a= t.split(' ')
b= s.split(' ')

for i in range(len(a)):
    if a[i]!= b[i]:
        print(a[i],'---',b[i])

## IOB tagger data preparation

In [ ]:
def findTag(SpeechLine,tagline,startpos_speech,len_speech):
    prev_words= SpeechLine[:startpos_speech].split(' ')
 
    n= len(prev_words)
    tagline[n]='B'
    for i in range(1,len_speech,1):
        tagline[(n+i)] = 'I'        
    
    return tagline

In [ ]:
file = 'Comparison-files/comp-VS-1035-ESQMIA-J7374393.txt'

#step-1 make a single string of speech,text and eval from a whole text file
import re
with open(file,'r') as f:
    lines=f.readlines()

linecount =0

for line in lines:
    linecount +=1

print(linecount)
SpeechLine = ''
TextLine = ''
EvalLine = ''

i=16          #avoid first 16 lines

while True:

    if (i+3) < linecount: 

        TextLine += lines[i][8:]
        SpeechLine += lines[i+1][8:]
        EvalLine += lines[i+2][8:]

        i +=4
    else: break   
SpeechLine = re.sub('\n','', SpeechLine)
TextLine = re.sub('\n','',TextLine)
EvalLine = re.sub('\n','',EvalLine)






#step-2 find location of 'I',"S" inside evalLIne

template = []

for i in range(len(EvalLine)):
    if EvalLine[i] == 'I' or EvalLine[i] == 'S':
        template.append(i)



#step-3: start with a 'S' find next 'S' and check if it is the next word of sppechline. if yes then find next else it's the end 



#textData = []
#speechData = []



words= SpeechLine.split(' ')
tagline= []
for word in words:
    tagline.append('O')


i=0

while True:

    if EvalLine[template[i]] == 'S':

        startpos_text = template[i]
        len_text= 1

        curr_s = i
        b= i+1

        while True:
            if b > (len(template)-1):
                break
            elif EvalLine[template[b]] == 'S':

                next_s= b

                #slice_from_curr_s
                slices_from_curr = SpeechLine[template[curr_s]:].split(' ')
                slices_from_next =  SpeechLine[template[next_s]:].split(' ')


                if slices_from_curr[1]== slices_from_next[0] :
                    curr_s = b
                    len_text +=1
                    b +=1
                    continue
                else:
                    break
            else:
                break


        a=i-1
        while True:
            if a < 0:
                break

            elif EvalLine[template[a]] == 'S':
                break


            elif EvalLine[template[a]] == 'I':
                a= a-1
                continue

        startpos_speech = template[(a+1)]

        len_speech = b-a-1

        #t= findText(TextLine,startpos_text,len_text)
        #s= findText(SpeechLine,startpos_speech, len_speech)

        tagline = findTag(SpeechLine,tagline,startpos_speech,len_speech)
        #textData.append(t)
        #speechData.append(s)    

        i = b

    else:
        i +=1

    if i > (len(template)-1):
        break

In [ ]:
print(tagline)

In [ ]:
print(SpeechLine)

### Creating\ text files with same name in other directory

In [ ]:
import os
#src_dir = '../formatter/all_speech'
tar_dir =  '../formatter/all_text_cleaned'
curr_dir = '/home/nayan/formatter/all_text'

with os.scandir(curr_dir) as entries:
    for entry in entries:
        chk = str(entry)
       
        chk= chk[11:-2]
       
        nam = os.path.join(tar_dir,chk)
        print(nam)
        
       
        #f = open(nam, 'w') 
        
        
      
        
    print('Done!!!')


In [ ]:
#file= 'all_text/text-form-VS-1035-ESQMIA-J7374393.txt'
#output_file = 'all_text_cleaned/text-form-VS-1035-ESQMIA-J7374393.txt'
#! ./MetalTiger transform -config metaltiger-master/sample_models/lm_clean/en_US/text_cleaning.toml -input $file \
#                    >$output_file

In [ ]:
#! file='all_text/text-form-VS-1035-ESQMIA-J7374393.txt'; outputfile='all_speech/speech.txt'; ./MetalTiger transform -config metaltiger-master/sample_models/lm_clean/en_US/spokenform.toml -input $file \
#     > $outputfile

In [ ]:
#! ./MetalTiger transform -config metaltiger-master/sample_models/lm_clean/en_US/spokenform.toml -input $file> $output_file

In [ ]:
a= '          I         I   S        S       '
b= 'september FIFTEENTH TWO THOUSAND THREE and'
c= 'september ********* *** 15TH     2003'
count =0
for i in range(len(a)):
    #print(i,a[i])
    if a[i]== 'I' or a[i]== 'S':
        if a[i]=='I' and i > 0:
            if a[i-1] != 'I' : 
        count +=1
    
as= ''
at= ''

speech= b[10:]
speech= speech.split(' ')
for i in range(count):
    at= at+ ' '+speech[i]
   
print(at)
